In [1]:
import os
import yaml
import json
import pandas as pd

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [4]:
import wombat.bsub as bsub

load sample data

In [5]:
m = json.load(open('../tests/data/tinjasmine/C3L-01232.json'))
m

{'case_id': 'C3L-01232',
 'disease': 'GBM',
 'WXS': {'normal': {'id': 'C3L-01232.WXS.N.hg38',
   'barcode': 'CPT0240430002',
   'filepath': '/storage1/fs1/m.wyczalkowski/Active/Primary/CPTAC3.share/CPTAC3-GDC/GDC_import/data/a65f1db8-170a-4ec2-a20b-e6deff964e2c/54352f31-de32-4a52-b0c0-1ab882496fc3_wxs_gdc_realn.bam'},
  'tumor': {'id': 'C3L-01232.WXS.T.hg38',
   'barcode': 'CPT0239440006',
   'filepath': '/storage1/fs1/m.wyczalkowski/Active/Primary/CPTAC3.share/CPTAC3-GDC/GDC_import/data/6da28b7b-b148-4c49-97ee-d4587b058a46/218349a8-16e2-4bcd-9c9a-690bc5152441_wxs_gdc_realn.bam'}},
 'WGS': {'normal': {'id': 'C3L-01232.WGS.N.hg38',
   'filepath': '/storage1/fs1/m.wyczalkowski/Active/Primary/CPTAC3.share/CPTAC3-GDC/GDC_import/data/c7777e0c-2447-4d47-aa47-e65bed9a8e47/54352f31-de32-4a52-b0c0-1ab882496fc3_wgs_gdc_realn.bam'},
  'tumor': {'id': 'C3L-01232.WGS.T.hg38',
   'filepath': '/storage1/fs1/m.wyczalkowski/Active/Primary/CPTAC3.share/CPTAC3-GDC/GDC_import/data/7794e4ec-ebc2-40ca-9e51-

make test input yaml

In [18]:
template = yaml.safe_load(open('/home/estorrs/TinJasmine/cwl/template.TinJasmine.yaml'))
template

{'sample_barcode': 'a_string',
 'centromere': {'class': 'File', 'path': 'a/file/path'},
 'chrlist': {'class': 'File', 'path': 'a/file/path'},
 'Canonical_BED': {'class': 'File', 'path': 'a/file/path'},
 'assembly': 'a_string',
 'vep_cache_version': 'a_string',
 'vep_cache_gz': {'class': 'File', 'path': 'a/file/path'},
 'ROI_BED': {'class': 'File', 'path': 'a/file/path'},
 'varscan_filter_config': {'class': 'File', 'path': 'a/file/path'},
 'pindel_filter_config': {'class': 'File', 'path': 'a/file/path'},
 'pindel_config_template': {'class': 'File', 'path': 'a/file/path'},
 'gatk_filter_config': {'class': 'File', 'path': 'a/file/path'},
 'reference': {'class': 'File', 'path': 'a/file/path'},
 'bam': {'class': 'File', 'path': 'a/file/path'}}

In [7]:
t2 = yaml.safe_load(open('/home/estorrs/TinJasmine/testing/cromwell-simple-compute1/dat/TinJasmine.C3L-00081.yaml'))
t2

{'centromere': {'class': 'File',
  'path': '/storage1/fs1/dinglab/Active/Resources/Pipelines/TinJasmine/TinJasmine_etc/centromere/ucsc-centromere.GRCh38.bed'},
 'chrlist': {'class': 'File',
  'path': '/storage1/fs1/dinglab/Active/Resources/Pipelines/TinJasmine/TinJasmine_etc/chrlist/GRCh38.d1.vd1.chrlist-reordered.txt'},
 'pindel_config_template': {'class': 'File',
  'path': '/storage1/fs1/dinglab/Active/Resources/Pipelines/TinJasmine/TinJasmine_etc/pindel_config_template/pindel_germline_filter_config.ini'},
 'bam': {'class': 'File',
  'path': '/storage1/fs1/m.wyczalkowski/Active/Primary/CPTAC3.share/CPTAC3-GDC/GDC_import/data/16477a75-08b9-4c77-9117-30dbb59d9f84/499d2ebb-679e-4720-8b86-3c989769d905_gdc_realn.bam'},
 'normal_barcode': 'barcode',
 'Canonical_BED': {'class': 'File',
  'path': '/storage1/fs1/dinglab/Active/Resources/Pipelines/TinJasmine/TinJasmine_etc/Canonical_BED/GRCh38.callRegions.bed'},
 'assembly': 'GRCh38',
 'vep_cache_version': 99,
 'vep_cache_gz': {'class': 'File'

In [9]:
t2.keys()

dict_keys(['centromere', 'chrlist', 'pindel_config_template', 'bam', 'normal_barcode', 'Canonical_BED', 'assembly', 'vep_cache_version', 'vep_cache_gz', 'ROI_BED', 'varscan_filter_config', 'pindel_filter_config', 'gatk_filter_config', 'reference'])

In [8]:
t2['vep_cache_gz']['path'] = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/vep/v99/vep-cache.99_GRCh38.tar.gz'
t2['reference']['path'] = '/storage1/fs1/dinglab/Active/Resources/References/GRCh38.d1.vd1/GRCh38.d1.vd1.fa'

In [12]:
t2['bam']['path'] = m['WXS']['normal']['filepath']
t2['sample_barcode'] = m['WXS']['normal']['barcode']

# sample template hasn't been updated to new cwl name in some versions
if 'normal_barcode' in t2:
    t2.pop('normal_barcode')

In [13]:
t2

{'centromere': {'class': 'File',
  'path': '/storage1/fs1/dinglab/Active/Resources/Pipelines/TinJasmine/TinJasmine_etc/centromere/ucsc-centromere.GRCh38.bed'},
 'chrlist': {'class': 'File',
  'path': '/storage1/fs1/dinglab/Active/Resources/Pipelines/TinJasmine/TinJasmine_etc/chrlist/GRCh38.d1.vd1.chrlist-reordered.txt'},
 'pindel_config_template': {'class': 'File',
  'path': '/storage1/fs1/dinglab/Active/Resources/Pipelines/TinJasmine/TinJasmine_etc/pindel_config_template/pindel_germline_filter_config.ini'},
 'bam': {'class': 'File',
  'path': '/storage1/fs1/m.wyczalkowski/Active/Primary/CPTAC3.share/CPTAC3-GDC/GDC_import/data/a65f1db8-170a-4ec2-a20b-e6deff964e2c/54352f31-de32-4a52-b0c0-1ab882496fc3_wxs_gdc_realn.bam'},
 'Canonical_BED': {'class': 'File',
  'path': '/storage1/fs1/dinglab/Active/Resources/Pipelines/TinJasmine/TinJasmine_etc/Canonical_BED/GRCh38.callRegions.bed'},
 'assembly': 'GRCh38',
 'vep_cache_version': 99,
 'vep_cache_gz': {'class': 'File',
  'path': '/storage1/fs1

In [15]:
yaml.safe_dump(t2, open('../tests/data/tinjasmine/C3L-01232_inputs.yaml', 'w'))

make cromwell compute1 config

In [19]:
workflow_root = '/scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/tinjasmine/C3L-01232'
output_fp = '../tests/data/tinjasmine/C3L-01232.cromwell-config-db.compute1.dat'
bsub.save_compute1_cromwell_template(workflow_root, output_fp)

make execution script

In [25]:
TEST_DIR_ROOT = '/storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/tinjasmine'
TOOL_ROOT = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline/submodules/TinJasmine/'

dconfig = os.path.join(TEST_DIR_ROOT, 'C3L-01232.cromwell-config-db.compute1.dat')
cwl_fp = os.path.join(TOOL_ROOT, 'cwl', 'TinJasmine.cwl')
inputs_fp = os.path.join(TEST_DIR_ROOT, 'C3L-01232_inputs.yaml')
volumes = [TEST_DIR_ROOT, TOOL_ROOT, workflow_root, '/storage1/fs1/dinglab/Active', '/storage1/fs1/m.wyczalkowski/Active']

In [26]:
args = bsub.DEFAULT_ARGS
args['group_name'] = 'test_cromwell'
args

{'mem': 10,
 'max_mem': None,
 'docker': 'python:3.8',
 'queue': 'dinglab',
 'group': 'compute-dinglab',
 'group_name': 'test_cromwell',
 'n_concurrent': 10,
 'interactive': False,
 'username': 'estorrs'}

In [27]:
start_commands, run_command = bsub.cromwell_commands(dconfig, cwl_fp, inputs_fp, args, volumes)
filepath = os.path.join('../tests/data/tinjasmine/start_server.sh')
bsub.write_command_file(start_commands, filepath)
filepath = os.path.join('../tests/data/tinjasmine/run_job.sh')
bsub.write_command_file([run_command], filepath)

In [28]:
for c in start_commands: print(c)

source /opt/ibm/lsfsuite/lsf/conf/lsf.conf
export LSF_DOCKER_NETWORK=host
export LSF_DOCKER_VOLUMES="/storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/tinjasmine:/storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/tinjasmine /storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline/submodules/TinJasmine/:/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline/submodules/TinJasmine/ /scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/tinjasmine/C3L-01232:/scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/tinjasmine/C3L-01232 /storage1/fs1/dinglab/Active:/storage1/fs1/dinglab/Active /storage1/fs1/m.wyczalkowski/Active:/storage1/fs1/m.wyczalkowski/Active"
bgadd -L 10 /estorrs/test_cromwell
bsub -q dinglab-interactive -G compute-dinglab -a 'docker(mwyczalkowski/cromwell-runner)' -Is '/bin/bash'


In [29]:
print(run_command)

/usr/bin/java -Dconfig.file=/storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/tinjasmine/C3L-01232.cromwell-config-db.compute1.dat -Djavax.net.ssl.trustStorePassword=changeit -Djavax.net.ssl.trustStore=/gscmnt/gc2560/core/genome/cromwell/cromwell.truststore -jar /usr/local/cromwell/cromwell-47.jar run -t cwl -i /storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/tinjasmine/C3L-01232_inputs.yaml /storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline/submodules/TinJasmine/cwl/TinJasmine.cwl
